<a href="https://colab.research.google.com/github/hieu1999210/Insincere-question-classification/blob/master/group_Relu_Glove_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers




Using TensorFlow backend.


In [0]:
!pip install h5py pyyaml
!pip install tf_nightly

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
cd "drive/My Drive"


/content/drive/My Drive


In [0]:
path_train = "data/train.csv"
path_test = "data/test.csv"
train = pd.read_csv(path_train)
test = pd.read_csv(path_test)
# Dataset is now stored in a Pandas Dataframe

In [0]:
print(train.shape)
print(test.shape)

(1306122, 3)
(375806, 2)


In [0]:
def build_vocab(sentences, verbose =  True):
    """
    :param sentences: list of list of words
    :return: dictionary of words and their count
    """
    vocab = {}
    for sentence in tqdm(sentences, disable = (not verbose)):
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

In [0]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
train_X = train["question_text"].progress_apply(lambda x: x.split()).values
vocab = build_vocab(train_X)
print({k: vocab[k] for k in list(vocab)[:5]})

100%|██████████| 1306122/1306122 [00:05<00:00, 232264.35it/s]

{'How': 261930, 'did': 33489, 'Quebec': 97, 'nationalists': 91, 'see': 9003}


In [0]:
embedding_file = 'embeddings/glove.840B.300d.txt'
def get_coefs(word, *arr):
  return word, np.asarray(arr, dtype='float32')

embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(embedding_file))



In [0]:
import operator
def check_coverage(vocab,embeddings_index):
    a = {}
    oov = {}
    k = 0
    i = 0
    for word in tqdm(vocab):
        try:
            a[word] = embeddings_index[word]
            k += vocab[word]
        except:

            oov[word] = vocab[word]
            i += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(a) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(k / (k + i)))
    sorted_x = sorted(oov.items(), key=operator.itemgetter(1))[::-1]

    return sorted_x

In [0]:
oov = check_coverage(vocab, embeddings_index)

100%|██████████| 508823/508823 [00:01<00:00, 484281.17it/s]


Found embeddings for 33.16% of vocab
Found embeddings for  88.16% of all text


In [0]:
oov[:10]

[('India?', 16384),
 ('it?', 12900),
 ("What's", 12425),
 ('do?', 8753),
 ('life?', 7753),
 ('you?', 6295),
 ('me?', 6202),
 ('them?', 6140),
 ('time?', 5716),
 ('world?', 5386)]

In [0]:
def clean_text(x):

    x = str(x)
    for punct in "/-'":
        x = x.replace(punct, ' ')
    for punct in '&?':
        x = x.replace(punct, f' {punct} ')
    for punct in '!.,"#$%\'()*+-/:;<=>@[\\]^_`{|}~' + '“”’':
        x = x.replace(punct, '')
    return x

In [0]:
train["question_text"] = train["question_text"].progress_apply(lambda x: clean_text(x))
train_X = train["question_text"].apply(lambda x: x.split())
vocab = build_vocab(train_X)

100%|██████████| 1306122/1306122 [00:05<00:00, 233951.74it/s]


In [0]:
oov = check_coverage(vocab, embeddings_index)

100%|██████████| 253529/253529 [00:00<00:00, 627321.13it/s]


Found embeddings for 71.40% of vocab
Found embeddings for  99.48% of all text


In [0]:
oov[:10]

[('Quorans', 858),
 ('Brexit', 493),
 ('cryptocurrencies', 481),
 ('Redmi', 379),
 ('OnePlus', 125),
 ('UCEED', 123),
 ('GDPR', 107),
 ('Blockchain', 107),
 ('demonetisation', 106),
 ('Coinbase', 104)]

In [0]:
train_X.shape
print(type(vocab))

<class 'dict'>


In [0]:
train, val = train_test_split(train, test_size = 0.1, random_state = 2019) 
embed_size = 300
max_features = 50000
maxlen = 100

## fill up the missing values
train_X = train["question_text"].fillna("_na_").values
val_X = val["question_text"].fillna("_na_").values
test_X = test["question_text"].fillna("_na_").values

#Tokenize the sentences
tokenizer = Tokenizer(num_words = max_features)
tokenizer.fit_on_texts(list(train_X))
train_X = tokenizer.texts_to_sequences(train_X)
val_X = tokenizer.texts_to_sequences(val_X)
# test_X = tokenizer.texts_to_sequences(test_X)

#padding
train_X = pad_sequences(train_X, maxlen=100)
val_X = pad_sequences(val_X, maxlen=100)

In [0]:
print(len(val_X))
print(len(train_X[0].shape))

1


In [0]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((50000, 300))
for word, index in tokenizer.word_index.items():
    if index > 50000 - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [0]:
train_y = train['target'].values
val_y = val['target'].values

In [0]:
import tensorflow as tf
import tensorflow_hub as hub
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(y_true * y_pred)
        possible_positives = K.sum(y_true)
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(y_true * y_pred)
        predicted_positives = K.sum(y_pred)
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    
    Y_pred = tf.cast(tf.less(0.3, y_pred), tf.float32)
    
    precision = precision(y_true, Y_pred)
    recall = recall(y_true, Y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding


def build_model():
    model_glove = Sequential()
    model_glove.add(Embedding(50000, 300, input_length=100, weights=[embedding_matrix], trainable=False))
    model_glove.add(Dropout(0.2))
    model_glove.add(Conv1D(64, 5, activation='relu'))
    model_glove.add(MaxPooling1D(pool_size=4))
    model_glove.add(LSTM(300))
    model_glove.add(Dense(1, activation='sigmoid'))
    model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=[f1])
    
    return model_glove

model_glove = build_model()

model_glove.summary()

W0706 05:46:54.996817 139766792431488 deprecation_wrapper.py:118] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0706 05:46:55.643271 139766792431488 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0706 05:46:55.700432 139766792431488 deprecation_wrapper.py:118] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0706 05:46:56.167027 139766792431488 deprecation_wrapper.py:118] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 300)          15000000  
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 300)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 96, 64)            96064     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 24, 64)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 300)               438000    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 301       
Total params: 15,534,365
Trainable params: 534,365
Non-trainable params: 15,000,000
__________________________________________________________

In [0]:
print(len(val_X))
print(len(val_y))
130613 - 130613%256

130613
130613


130560

In [0]:
import tensorflow as tf
# include the epoch in the file name. (uses `str.format`)
checkpoint_path = "data/training_glove/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # Save weights, every 5-epochs.
    period=1)


history = model_glove.fit(np.array(train_X[:9404288]), np.array(train_y[:9404288]), validation_data=(np.array(val_X[:130560]), val_y[:130560]), batch_size = 256, epochs = 6, callbacks = [cp_callback])

W0706 06:06:11.286353 139766792431488 callbacks.py:861] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


Train on 1175509 samples, validate on 130560 samples
Epoch 1/6
1175509/1175509 [==============================] - 3468s 3ms/step - loss: 0.1164 - f1: 0.6080 - val_loss: 0.1122 - val_f1: 0.6398

Epoch 00001: saving model to data/training_glove/cp-0001.ckpt
Epoch 2/6
1175509/1175509 [==============================] - 3649s 3ms/step - loss: 0.1079 - f1: 0.6363 - val_loss: 0.1084 - val_f1: 0.6508

Epoch 00002: saving model to data/training_glove/cp-0002.ckpt
Epoch 3/6
1175509/1175509 [==============================] - 3816s 3ms/step - loss: 0.1035 - f1: 0.6532 - val_loss: 0.1086 - val_f1: 0.6471

Epoch 00003: saving model to data/training_glove/cp-0003.ckpt
Epoch 4/6
1175509/1175509 [==============================] - 3817s 3ms/step - loss: 0.1002 - f1: 0.6633 - val_loss: 0.1066 - val_f1: 0.6528

Epoch 00004: saving model to data/training_glove/cp-0004.ckpt
Epoch 5/6
1175509/1175509 [==============================] - 3730s 3ms/step - loss: 0.0974 - f1: 0.6712 - val_loss: 0.1100 - val_f1: 0.

In [0]:
9404407-940407%256


9404288

In [0]:
predict_val_y = model_glove.predict(val_X, batch_size = 1024, verbose = 1)

130613/130613 [==============================] - 113s 866us/step


In [0]:
for thresh in np.arange(0.1, 0.901, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (predict_val_y>thresh).astype(int))))

F1 score at threshold 0.1 is 0.5902389217888503
F1 score at threshold 0.11 is 0.598319889664396
F1 score at threshold 0.12 is 0.6063502902014339
F1 score at threshold 0.13 is 0.6140045288277305
F1 score at threshold 0.14 is 0.6204693669313694
F1 score at threshold 0.15 is 0.6257563442608146
F1 score at threshold 0.16 is 0.6312164209946274
F1 score at threshold 0.17 is 0.6356299855390213
F1 score at threshold 0.18 is 0.6395646000946521
F1 score at threshold 0.19 is 0.6419433663935605
F1 score at threshold 0.2 is 0.6445607911576497
F1 score at threshold 0.21 is 0.6480117820324006
F1 score at threshold 0.22 is 0.6507628845484816
F1 score at threshold 0.23 is 0.6537474203452961
F1 score at threshold 0.24 is 0.6558980891719745
F1 score at threshold 0.25 is 0.6577264287187952
F1 score at threshold 0.26 is 0.6586788914357159
F1 score at threshold 0.27 is 0.6595072646873026
F1 score at threshold 0.28 is 0.6606382978723404
F1 score at threshold 0.29 is 0.6607565011820332
F1 score at threshold 0

In [0]:
!ls {checkpoint_dir}

cp-0001.ckpt  cp-0003.ckpt  cp-0005.ckpt
cp-0002.ckpt  cp-0004.ckpt  cp-0006.ckpt


In [0]:
checkpoint_path = "data/training_glove/cp-0001.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)



In [0]:
model_glove.load_weights(checkpoint_path)
predict_val_y = model_glove.predict(val_X, batch_size = 1024, verbose = 1)

for thresh in np.arange(0.1, 0.901, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (predict_val_y>thresh).astype(int))))

130613/130613 [==============================] - 115s 880us/step
F1 score at threshold 0.1 is 0.5754442649434571
F1 score at threshold 0.11 is 0.5849739854318419
F1 score at threshold 0.12 is 0.5938741509675766
F1 score at threshold 0.13 is 0.6017171894165061
F1 score at threshold 0.14 is 0.6084892988269007
F1 score at threshold 0.15 is 0.6145466196731816
F1 score at threshold 0.16 is 0.6190876799401758
F1 score at threshold 0.17 is 0.6238435860753456
F1 score at threshold 0.18 is 0.6277634711627229
F1 score at threshold 0.19 is 0.6305565191535756
F1 score at threshold 0.2 is 0.6340258168616377
F1 score at threshold 0.21 is 0.6381255127153405
F1 score at threshold 0.22 is 0.6408131352619234
F1 score at threshold 0.23 is 0.6434239762988044
F1 score at threshold 0.24 is 0.644364495516297
F1 score at threshold 0.25 is 0.6457981131046518
F1 score at threshold 0.26 is 0.6465774632312286
F1 score at threshold 0.27 is 0.647975776606482
F1 score at threshold 0.28 is 0.6470120427175642
F1 score

In [0]:
checkpoint_path_2 = "data/training_glove/cp-0002.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path_2)
model_glove.load_weights(checkpoint_path)
predict_val_y = model_glove.predict(val_X, batch_size = 1024, verbose = 1)

for thresh in np.arange(0.1, 0.901, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (predict_val_y>thresh).astype(int))))

130613/130613 [==============================] - 113s 867us/step
F1 score at threshold 0.1 is 0.5754442649434571
F1 score at threshold 0.11 is 0.5849739854318419
F1 score at threshold 0.12 is 0.5938741509675766
F1 score at threshold 0.13 is 0.6017171894165061
F1 score at threshold 0.14 is 0.6084892988269007
F1 score at threshold 0.15 is 0.6145466196731816
F1 score at threshold 0.16 is 0.6190876799401758
F1 score at threshold 0.17 is 0.6238435860753456
F1 score at threshold 0.18 is 0.6277634711627229
F1 score at threshold 0.19 is 0.6305565191535756
F1 score at threshold 0.2 is 0.6340258168616377
F1 score at threshold 0.21 is 0.6381255127153405
F1 score at threshold 0.22 is 0.6408131352619234
F1 score at threshold 0.23 is 0.6434239762988044
F1 score at threshold 0.24 is 0.644364495516297
F1 score at threshold 0.25 is 0.6457981131046518
F1 score at threshold 0.26 is 0.6465774632312286
F1 score at threshold 0.27 is 0.647975776606482
F1 score at threshold 0.28 is 0.6470120427175642
F1 score

In [0]:
checkpoint_path_3 = "data/training_glove/cp-0003.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path_3)
model_glove.load_weights(checkpoint_path)
predict_val_y = model_glove.predict(val_X, batch_size = 1024, verbose = 1)

for thresh in np.arange(0.1, 0.901, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (predict_val_y>thresh).astype(int))))

130613/130613 [==============================] - 113s 866us/step
F1 score at threshold 0.1 is 0.5754442649434571
F1 score at threshold 0.11 is 0.5849739854318419
F1 score at threshold 0.12 is 0.5938741509675766
F1 score at threshold 0.13 is 0.6017171894165061
F1 score at threshold 0.14 is 0.6084892988269007
F1 score at threshold 0.15 is 0.6145466196731816
F1 score at threshold 0.16 is 0.6190876799401758
F1 score at threshold 0.17 is 0.6238435860753456
F1 score at threshold 0.18 is 0.6277634711627229
F1 score at threshold 0.19 is 0.6305565191535756
F1 score at threshold 0.2 is 0.6340258168616377
F1 score at threshold 0.21 is 0.6381255127153405
F1 score at threshold 0.22 is 0.6408131352619234
F1 score at threshold 0.23 is 0.6434239762988044
F1 score at threshold 0.24 is 0.644364495516297
F1 score at threshold 0.25 is 0.6457981131046518
F1 score at threshold 0.26 is 0.6465774632312286
F1 score at threshold 0.27 is 0.647975776606482
F1 score at threshold 0.28 is 0.6470120427175642
F1 score

In [0]:
checkpoint_path_4 = "data/training_glove/cp-0004.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path_4)
model_glove.load_weights(checkpoint_path)
predict_val_y = model_glove.predict(val_X, batch_size = 1024, verbose = 1)

for thresh in np.arange(0.1, 0.901, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (predict_val_y>thresh).astype(int))))

130613/130613 [==============================] - 113s 869us/step
F1 score at threshold 0.1 is 0.5754442649434571
F1 score at threshold 0.11 is 0.5849739854318419
F1 score at threshold 0.12 is 0.5938741509675766
F1 score at threshold 0.13 is 0.6017171894165061
F1 score at threshold 0.14 is 0.6084892988269007
F1 score at threshold 0.15 is 0.6145466196731816
F1 score at threshold 0.16 is 0.6190876799401758
F1 score at threshold 0.17 is 0.6238435860753456
F1 score at threshold 0.18 is 0.6277634711627229
F1 score at threshold 0.19 is 0.6305565191535756
F1 score at threshold 0.2 is 0.6340258168616377
F1 score at threshold 0.21 is 0.6381255127153405
F1 score at threshold 0.22 is 0.6408131352619234
F1 score at threshold 0.23 is 0.6434239762988044
F1 score at threshold 0.24 is 0.644364495516297
F1 score at threshold 0.25 is 0.6457981131046518
F1 score at threshold 0.26 is 0.6465774632312286
F1 score at threshold 0.27 is 0.647975776606482
F1 score at threshold 0.28 is 0.6470120427175642
F1 score

In [0]:
checkpoint_path_5 = "data/training_glove/cp-0005.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path_5)
model_glove.load_weights(checkpoint_path)
predict_val_y = model_glove.predict(val_X, batch_size = 1024, verbose = 1)

for thresh in np.arange(0.1, 0.901, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (predict_val_y>thresh).astype(int))))

130613/130613 [==============================] - 114s 870us/step
F1 score at threshold 0.1 is 0.5754442649434571
F1 score at threshold 0.11 is 0.5849739854318419
F1 score at threshold 0.12 is 0.5938741509675766
F1 score at threshold 0.13 is 0.6017171894165061
F1 score at threshold 0.14 is 0.6084892988269007
F1 score at threshold 0.15 is 0.6145466196731816
F1 score at threshold 0.16 is 0.6190876799401758
F1 score at threshold 0.17 is 0.6238435860753456
F1 score at threshold 0.18 is 0.6277634711627229
F1 score at threshold 0.19 is 0.6305565191535756
F1 score at threshold 0.2 is 0.6340258168616377
F1 score at threshold 0.21 is 0.6381255127153405
F1 score at threshold 0.22 is 0.6408131352619234
F1 score at threshold 0.23 is 0.6434239762988044
F1 score at threshold 0.24 is 0.644364495516297
F1 score at threshold 0.25 is 0.6457981131046518
F1 score at threshold 0.26 is 0.6465774632312286
F1 score at threshold 0.27 is 0.647975776606482
F1 score at threshold 0.28 is 0.6470120427175642
F1 score

In [0]:
checkpoint_path_6 = "data/training_glove/cp-0006.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path_6)
model_glove.load_weights(checkpoint_path)
predict_val_y = model_glove.predict(val_X, batch_size = 1024, verbose = 1)

for thresh in np.arange(0.1, 0.901, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (predict_val_y>thresh).astype(int))))

130613/130613 [==============================] - 114s 873us/step
F1 score at threshold 0.1 is 0.5754442649434571
F1 score at threshold 0.11 is 0.5849739854318419
F1 score at threshold 0.12 is 0.5938741509675766
F1 score at threshold 0.13 is 0.6017171894165061
F1 score at threshold 0.14 is 0.6084892988269007
F1 score at threshold 0.15 is 0.6145466196731816
F1 score at threshold 0.16 is 0.6190876799401758
F1 score at threshold 0.17 is 0.6238435860753456
F1 score at threshold 0.18 is 0.6277634711627229
F1 score at threshold 0.19 is 0.6305565191535756
F1 score at threshold 0.2 is 0.6340258168616377
F1 score at threshold 0.21 is 0.6381255127153405
F1 score at threshold 0.22 is 0.6408131352619234
F1 score at threshold 0.23 is 0.6434239762988044
F1 score at threshold 0.24 is 0.644364495516297
F1 score at threshold 0.25 is 0.6457981131046518
F1 score at threshold 0.26 is 0.6465774632312286
F1 score at threshold 0.27 is 0.647975776606482
F1 score at threshold 0.28 is 0.6470120427175642
F1 score